# imports

In [23]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

# def

In [59]:
def getMoviecode(year=2021, max_page=40):
    
    """연도별 영화코드 수집"""
    
    base_url = 'https://movie.naver.com/movie/sdb/browsing/bmovie.naver?open=' + str(year) + '&page='
    movie_code_list = []
    
    for page in range(max_page + 1):
        
        url = base_url + str(page)
        response = requests.get(url)
        
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'lxml')
        else:
            print('error', response.status_code)
            break
        
        list = soup.select('#old_content > ul > li')
        for movie in range(len(list)):
            code = list[movie].select_one('a').get('href').split('=')[1]
            movie_code_list.append(code)
    
    return movie_code_list

In [202]:
def getSynopsis(code=code):
    
    """영화 페이지에서 줄거리 가져오기"""
    
    url = 'https://movie.naver.com/movie/bi/mi/basic.naver?code=' + code
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'lxml')
    else:
        print('error', response.status_code)
    
    synopsis_list = []
    title   = soup.select_one('#content > div.article > div.mv_info_area > div.mv_info > h3 > a:nth-of-type(1)').text
    synopsis_list.append(title)
    
    try:
        content = soup.select_one('#content > div.article > div.section_group.section_group_frst > div:nth-of-type(1) > div > div > p').text
        brief   = soup.select_one('#content > div.article > div.section_group.section_group_frst > div:nth-of-type(1) > div > div > h5').text
        synopsis_list.append(content)
        synopsis_list.append(brief)
    except:
        synopsis_list.append(np.nan)
        synopsis_list.append(np.nan)
    
    return synopsis_list

In [203]:
getSynopsis('191559')

['듄',
 "10191년, 아트레이데스 가문의 후계자인 폴(티모시 샬라메)은 시공을 초월한 존재이자\r\xa0전 우주를 구원할 예지된 자의 운명을 타고났다.\r\xa0그리고 어떤 계시처럼 매일 꿈에서 아라키스 행성에 있는 한 여인을 만난다.\r\xa0모래언덕을 뜻하는 '듄'이라 불리는 아라키스는 물 한 방울 없는 사막이지만\r\xa0우주에서 가장 비싼 물질인 신성한 환각제 스파이스의 유일한 생산지로 이를 차지하기 위한 전쟁이 치열하다.\r\xa0황제의 명령으로 폴과 아트레이데스 가문은 죽음이 기다리는 아라키스로 향하는데…\r\xa0\r\xa0위대한 자는 부름에 응답한다, 두려움에 맞서라, 이것은 위대한 시작이다!",
 '“듄을 지배하는 자가 우주를 지배한다!”']

# 영화 코드 가져오기

In [162]:
movie_codes = getMoviecode(year=2021, max_page=40)

In [181]:
len(movie_codes), movie_codes[:10]

(803,
 ['99702',
  '148043',
  '204776',
  '199665',
  '201842',
  '198765',
  '202533',
  '200743',
  '174831',
  '195072'])

# 영화 시놉시스 데이터프레임 구성

In [205]:
movie_list = []
for code in movie_codes:
    movie_list.append(getSynopsis(code=code))

In [206]:
len(movie_list)

803

In [215]:
df = pd.DataFrame(movie_list, columns=['제목', '줄거리', '요약'])
df['개봉년도'] = '2021'
df = df[['제목', '개봉년도', '요약', '줄거리']]

In [233]:
df['요약'] = df['요약'].str.replace('\r', ' ')
df['요약'] = df['요약'].str.replace('\n', '')
df['요약'] = df['요약'].str.replace('\xa0', '')

df['줄거리'] = df['줄거리'].str.replace('\r', ' ')
df['줄거리'] = df['줄거리'].str.replace('\n', '')
df['줄거리'] = df['줄거리'].str.replace('\xa0', '')

In [234]:
df.head(100)

,제목,개봉년도,요약,줄거리
0,007 노 타임 투 다이,2021,NaN,NaN
1,117편의 러브레터,2021,NaN,NaN
2,120시간: 로스트 인 아웃백,2021,NaN,NaN
3,1984 최동원,2021,NaN,NaN
4,"2021 스웨그 에이지 : 외쳐, 조선!",2021,‘시조’가 국가의 이념인 상상 속의 ‘조선’,삶의 고단함과 역경을 시조 속에 담아 훌훌 털어버렸던 백성들은 역모 사건으로 시조 ...
...,...,...,...,...
95,극장판 도라에몽: 진구의 신공룡,2021,도라에몽 50주년 기념대작! 오리지널 스토리로 돌아온 진구와 쌍둥이 공룡의 대모험!,진구는 공룡 엑스포 화석 발굴 체험에서 발견한 화석을 공룡알이라고 굳게 믿는다. 도...
96,극장판 마이 리틀 포니: 새로운 희망,2021,"함께 할 때 가장 빛나는 우리! 반짝이는 희망을 찾아, 임무를 시작해볼까?","포니, 페가수스, 유니콘이 함께 조화를 이루며 살던 이퀘스트리아. 하지만 평화가 사..."
97,극장판 바다 탐험대 옥토넛 : 불의 고리 대폭발,2021,"바닷속 화산이 폭발했다! 옥토넛, 출동!",평화로운 일상을 보내고 있던 옥토넛은태평양을 둘러싼 ‘불의 고리’에서 화산 폭발이 ...
98,극장판 짱구는 못말려: 격돌! 낙서왕국과 얼추 네 명의 용사들,2021,NaN,NaN


In [235]:
df.to_csv('./movie_synopsis.csv')